# TODO
#### Klassen protecten
```
import AutoMLWrapper.automlwrapper as aw
aw.Configuration.Configuration
```

#### Erweiterung um Bibliotheken
-  @__library.setter
    - a) Global/GlobalConfig.yaml
        - lib-names
        - type mappings
    - b) directory durchsuchen 
- import in AutoMLWrapper anpassen
- SEDAR automl.py config locations anpassen

### custom proprocessing
- typen checken
- die wrapper könnten property methods implementieren, die die Rückgabe eines festen samples prüfen

### custom parameters


### 2+ problem types


## mlflow
- eigene funktion für mlflow tracking
- user übergibt dict mit workspace, ...

In [1]:
import pandas as pd
#from automlwrapper import AutoMLWrapper
from AutoMLWrapper.automlwrapper import AutoMLWrapper


2023-11-29 11:45:04.250325: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 11:45:04.326755: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-29 11:45:04.328256: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 11:45:05.433728: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/mibbels/miniconda3/envs/automl_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


In [2]:
df = pd.read_csv('data/glass.csv')
user_hp = {'num_trials': 4, 'epochs':20, 'time_limit': 40, 'memory_limit':7000, 'testtest': 123}


leaderboard(detailed=True) -> dataframe
- type: random_forest
- cost: validation loss
- duration: ?
- data_preprocessoers : name sklearn class?
. feature_preprocessors : name sklearn class


show_models()
- model_id, rank, cost
- feature_preprocessor: autosklearn object
- data_preprocessor: autosklearn object
 
Alle Parameter res[3]['sklearn_classifier'].get_params()

Modell res[3]['sklearn_classifier']


In [3]:
ask = AutoMLWrapper('autosklearn')
ask.SetOutputDirectory('test/ask/6')

if 0:
    ask.Train(data=df, target_column='Type',
        task_type='classification',
        data_type='tabular',
        problem_type='multiclass',
        hyperparameters=user_hp
    )

## multimodal
content in "fit(save_path)"
    - asset.json - //autogluon info
    - config.yaml model+hp
    - hparams.yaml
    zip + pkl files

fit_summary()
    {'val_accuracy': 0.956250011920929, 'training_time': 314.0019574165344}

hpo
    ... !!!

## tabular
content in TabularPredictor(save_path)
    - pkl files
    
fit_summary()
    - performance (kein direktes ranking)
    - hp
    
## timeseries
content in TimeSeriesPredictor(save_path)
    - pkl files

leaderboard()
    - model, score

fit_summary()
    - performance, path, time

info()
    






In [4]:
agl = AutoMLWrapper('autogluon')
agl.SetOutputDirectory('test/agl')
if 0:
    agl.Train(data=df, target_column='Type',
       task_type='classification',
       data_type='tabular',
       problem_type='multiclass',
       hyperparameters=user_hp
   )


In [5]:
akr = AutoMLWrapper('autokeras')
akr.SetOutputDirectory('test/akr/1')
if 1:
    akr.Train(data=df, target_column='Type',
        task_type='classification',
        data_type='tabular',
        problem_type='multiclass',
        hyperparameters=user_hp
    )

Reloading Tuner from test/akr/1/structured_data_classifier/tuner0.json
Epoch 1/20
7/7 [==============================] - 1s 3ms/step - loss: 2.0891 - accuracy: 0.1682
Epoch 2/20
7/7 [==============================] - 0s 3ms/step - loss: 1.9335 - accuracy: 0.1963
Epoch 3/20
7/7 [==============================] - 0s 3ms/step - loss: 1.8441 - accuracy: 0.2570
Epoch 4/20
7/7 [==============================] - 0s 3ms/step - loss: 1.7985 - accuracy: 0.2290
Epoch 5/20
7/7 [==============================] - 0s 3ms/step - loss: 1.6782 - accuracy: 0.3271
Epoch 6/20
7/7 [==============================] - 0s 3ms/step - loss: 1.6587 - accuracy: 0.3131
Epoch 7/20
7/7 [==============================] - 0s 2ms/step - loss: 1.6961 - accuracy: 0.2804
Epoch 8/20
7/7 [==============================] - 0s 3ms/step - loss: 1.5909 - accuracy: 0.3458
Epoch 9/20
7/7 [==============================] - 0s 3ms/step - loss: 1.5777 - accuracy: 0.3458
Epoch 10/20
7/7 [==============================] - 0s 3ms/step - 

INFO:tensorflow:Assets written to: test/akr/1/structured_data_classifier/best_model/assets


# MLFlow
~/miniconda3/envs/automl_env/bin/pip

In [6]:
import mlflow
#experiment_id = mlflow.create_experiment('test_automl', artifact_location='/home/mibbels/sedar-masterarbeit/sedar/sedar/automl/tmp/artifacts')


experiment_id = 5
remote_server_uri = "http://localhost:6798"
mlflow.set_tracking_uri(remote_server_uri) 
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='/home/mibbels/sedar-masterarbeit/sedar/sedar/automl/tmp/artifacts', creation_time=1701189486674, experiment_id='5', last_update_time=1701189486674, lifecycle_stage='active', name='test_automl', tags={}>

In [11]:
out = akr.Output()

In [12]:
out

[{'tags': {},
  'params': {'epochs': 20,
   'optimizer_name': 'Adam',
   'optimizer_weight_decay': None,
   'optimizer_clipnorm': None,
   'optimizer_global_clipnorm': None,
   'optimizer_clipvalue': None,
   'optimizer_use_ema': False,
   'optimizer_ema_momentum': 0.99,
   'optimizer_ema_overwrite_frequency': None,
   'optimizer_jit_compile': False,
   'optimizer_is_legacy_optimizer': False,
   'optimizer_learning_rate': 0.001,
   'optimizer_beta_1': 0.9,
   'optimizer_beta_2': 0.999,
   'optimizer_epsilon': 1e-07,
   'optimizer_amsgrad': False},
  'metrics': {'loss': 1.3385281562805176, 'accuracy': 0.4532710313796997},
  'artifacts': {},
  'model': <keras.src.engine.functional.Functional at 0x7fdff4203100>}]

In [13]:
akr.MlflowUploadBest({})
#ask.MlflowUploadTopN(3, {})

{'tags': {}, 'params': {'epochs': 20, 'optimizer_name': 'Adam', 'optimizer_weight_decay': None, 'optimizer_clipnorm': None, 'optimizer_global_clipnorm': None, 'optimizer_clipvalue': None, 'optimizer_use_ema': False, 'optimizer_ema_momentum': 0.99, 'optimizer_ema_overwrite_frequency': None, 'optimizer_jit_compile': False, 'optimizer_is_legacy_optimizer': False, 'optimizer_learning_rate': 0.001, 'optimizer_beta_1': 0.9, 'optimizer_beta_2': 0.999, 'optimizer_epsilon': 1e-07, 'optimizer_amsgrad': False}, 'metrics': {'loss': 1.3385281562805176, 'accuracy': 0.4532710313796997}, 'artifacts': {}, 'model': <keras.src.engine.functional.Functional object at 0x7fe1253c0580>}


2023/11/29 11:47:10 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /tmp/tmptosrge6s/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmptosrge6s/model/data/model/assets
2023/11/29 11:47:16 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmptosrge6s/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.1']. Set logging level to DEBUG to see the full traceback.


In [10]:

with mlflow.start_run() as run:
       mlflow.set_tag("testing","automlwrapper")
       mlflow.log_param("dataset","glass")  

       for item in out:
        for key, value in item['tags'].items():
            mlflow.set_tag(key, str(value))
        for key, value in item['params'].items():
            mlflow.log_param(key, value)
        for key, value in item['metrics'].items():
            mlflow.log_metric(key, value)          


   